In [ ]:
##### IMPORTS #####
import os
import sys
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Lambda, GlobalAveragePooling2D

from pathlib import Path

from sklearn.model_selection import train_test_split

In [ ]:
food_dir = 'food/'
train_file = 'train_triplets.txt'
test_file = 'test_triplets.txt'
food_fnames = os.listdir(food_dir)
print('total number of images: ', len(os.listdir(food_dir)))

In [ ]:
##### SET PARAMETERS #####
TRAIN_ROWS = 59515 # Max = 59515
TEST_ROWS = 59544 # Max = 59544
RATIO = 0.2
RATIO_N_P = 0.3
NR_OF_IMAGES = 10000 # Max = 10'000
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_COLORS = 3
MARGIN = 0.3
TRAIN_BATCH = 32
TEST_BATCH = 72
VALID_BATCH = 128
BATCH_SIZE = 64
EPOCHS = 3

In [ ]:
##### LOCATION TO FILES #####
FOOD_LOC = 'food/'
TEST_LOC = 'test.txt'
TRAIN_LOC = 'train.txt'
TRAIN_TRIPLETS_LOC = 'train_triplets.txt'
TEST_TRIPLETS_LOC = 'test_triplets.txt'
SUBMISSION = 'submission.txt'

In [ ]:
##### HELPER FUNCTIONS #####

# Returns the distances between the points
def distance(a, p, n):
  dist_a_p = tf.sqrt(tf.reduce_sum(tf.square(a - p), axis=1, keepdims=True))
  dist_a_n = tf.sqrt(tf.reduce_sum(tf.square(a - n), axis=1, keepdims=True))
  dist_p_n = tf.reduce_sum(tf.square(a - n), axis=1, keepdims=True)
  return dist_a_p, dist_a_n, dist_p_n
# Returns the loss function of our model
def lossf(_, pred):
  dist_a_p, dist_a_n, dist_p_n = distance(pred[:,0], pred[:,1], pred[:,2])
  return tf.reduce_mean(tf.cast(tf.maximum(dist_a_p - (dist_a_n*(1-RATIO_N_P) + dist_p_n*RATIO_N_P) + MARGIN,0), tf.float32))
# Accuracy function for our model
def metrics(_, pred):
  dist_a_p, dist_a_n, dist_p_n = distance(pred[:,0], pred[:,1], pred[:,2])
  return tf.reduce_mean(tf.cast(tf.less_equal(dist_a_p, dist_a_n*(1-RATIO_N_P) + dist_p_n*RATIO_N_P), tf.float32))
# Classification function for our model
def classification(pred):
  dist_a_p, dist_a_n, dist_p_n = distance(pred[:,0], pred[:,1], pred[:,2])
  return tf.cast(tf.less_equal(dist_a_p, dist_a_n), tf.float32)
# Counts the amount of rows in a file, i.e. amount of triplets
def row_count(fname):
  count = 0
  with open(fname, 'r') as txtfile:
    for line in txtfile:
        count += 1
  return count
# Loads an image from the drive. Code from documentation but included image standardization and cast to our format
def load_modify_image(jpg):
  img = tf.io.read_file(FOOD_LOC + jpg + '.jpg')
  img = tf.image.decode_jpeg(img, channels=IMG_COLORS)
  img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
  img = tf.image.convert_image_dtype(img, dtype=tf.float32)
  img = preprocess_input(img)
  return img
# Loads the images for the corresponding triplet
def load_triplet(triplet):
  triplet = tf.strings.split(triplet)
  return (load_modify_image(triplet[0]), 
          load_modify_image(triplet[1]), 
          load_modify_image(triplet[2])), 0

In [ ]:
##### CREATE TRAIN_TEST DATA #####
# Important that the elements of these sets are disjoint to prevent overfitting
with open(TRAIN_TRIPLETS_LOC, 'r') as fin, open(TRAIN_LOC, 'w') as train, open(TEST_LOC, 'w') as test:
  rows = fin.readlines()    
  tr_row, te_row = train_test_split(
      list(dict.fromkeys([item for row in rows for item in row.split()])), 
      test_size=RATIO,
      random_state = 2)
  for row in rows:
    triplets = row.split()
    included_in_test = not bool(set(triplets) & set(te_row))
    included_in_train = not bool(set(triplets) & set(tr_row))
    if included_in_test or included_in_train:
      if included_in_test:
        train.write(row)
      elif included_in_train:
        test.write(row)
# Print the sizes of the sets and their ratio
ratio = np.round(row_count(TEST_LOC)/row_count(TRAIN_LOC)*100, 2)
print(row_count(TEST_LOC), row_count(TRAIN_LOC))
print("Ratio: ", ratio)
# Get the numbers of steps for each dataset later
steps_per_epoch = row_count(TRAIN_LOC) / float(TRAIN_BATCH)
validation_steps = row_count(TEST_LOC) / float(VALID_BATCH)
nr_test_steps = row_count(TEST_TRIPLETS_LOC)/float(TEST_BATCH)

In [ ]:
##### CREATE DATASETS #####
train_dataset = tf.data.TextLineDataset(TRAIN_LOC).map(load_triplet).shuffle(buffer_size=1000, seed = 2).repeat().batch(TRAIN_BATCH)
validation_dataset = tf.data.TextLineDataset(TEST_LOC).map(load_triplet).repeat().batch(VALID_BATCH)
test_dataset = tf.data.TextLineDataset(TEST_TRIPLETS_LOC).map(load_triplet).batch(TEST_BATCH)

In [ ]:
##### MODEL #####
base_model = InceptionV3(input_shape=(IMG_HEIGHT,IMG_WIDTH,IMG_COLORS), include_top=False)

in_img_a = Input(shape=(IMG_HEIGHT,IMG_WIDTH,IMG_COLORS))
in_img_p = Input(shape=(IMG_HEIGHT,IMG_WIDTH,IMG_COLORS))
in_img_n = Input(shape=(IMG_HEIGHT,IMG_WIDTH,IMG_COLORS))

for layer in base_model.layers:
    layer.trainable = False

func_vec_model = GlobalAveragePooling2D()(base_model.output)
func_vec_model = Dropout(0.33)(func_vec_model)
func_vec_model = Dense(256)(func_vec_model)
func_vec_model = Lambda(lambda vect: tf.math.l2_normalize(vect, axis=1))(func_vec_model)

jpg_enc = Model(base_model.input, func_vec_model)
layer_out = Lambda(lambda vects: tf.stack(vects, axis=1))([jpg_enc(in_img_a), jpg_enc(in_img_p), jpg_enc(in_img_n)])
model = Model((in_img_a, in_img_p, in_img_n), layer_out)
model.compile(optimizer=Adam(lr=0.0001), loss=lossf, metrics=[metrics])

In [ ]:
##### FIT MODEL ######
model.fit(train_dataset,
          steps_per_epoch=int(steps_per_epoch),
          batch_size=TRAIN_BATCH,
          epochs=1, 
          validation_data=validation_dataset,
          validation_steps=int(validation_steps),
          validation_batch_size=VALID_BATCH)

In [ ]:
##### MAKE PREDICTION #####
full_model = Model(model.input, layers.Lambda(classification)(model.output))
prediction = full_model.predict(test_dataset, verbose=1, steps=int(nr_test_steps))

In [ ]:
np.savetxt('submission_new.txt', prediction, fmt='%d')